pip install transformers torch torchvision

In [26]:
from huggingface_hub import login
import os
import pandas as pd
from dotenv import load_dotenv
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

In [27]:
dataset_name = "AndresR2909/youtube_transcriptions_summaries_2025_gpt4.1"
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_test = pd.read_parquet(f"hf://datasets/{dataset_name}/" + splits["test"])

In [28]:
load_dotenv()
login(token = os.environ.get("HF_TOKEN") )

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [24]:
# Load model directly

model = AutoModelForCausalLM.from_pretrained("AndresR2909/hf-llama-3.2-3b-finetuned_bnb_nf4_v2")

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 29.66it/s]


In [25]:

tokenizer = AutoTokenizer.from_pretrained("AndresR2909/hf-llama-3.2-3b-finetuned_bnb_nf4_v2")

In [31]:
def resumir_documento(texto, max_length=8128, min_length=10):
    # Tokenizar el texto de entrada
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length)
    # Generar el resumen usando el modelo
    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        early_stopping=True,#False
        pad_token_id=tokenizer.eos_token_id
    )
    # Decodificar solo los nuevos tokens generados (sin el input)
    input_length = inputs["input_ids"].shape[1]
    generated_tokens = summary_ids[0][input_length:]
    resumen = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return resumen

# Ejemplo de uso:
texto = df_test.loc[10, 'prompt']
print(resumir_documento(texto))

 Utiliza un lenguaje claro y conciso. No inventes datos; limita el análisis al contenido del texto.

**Informe de Análisis de Mercado**

- **Introducción:**  
El texto analiza la situación actual de los mercados financieros, con especial énfasis en la bolsa de Estados Unidos. Se discuten señales técnicas y fundamentales que sugieren cautela para los inversores en 2024, destacando factores como la euforia del mercado, altos niveles de valoraciones, desacoplamiento entre indicadores económicos y bursátiles, y el impacto de la economía estadounidense en las tasas de interés.

- **Puntos clave:**
  - La bolsa de Estados Unidos ha experimentado un fuerte rally desde mínimos, con un aumento del 5-6% en el índice Nasdaq y cerca de máximos.
  - Se observa euforia y optimismo excesivos en el mercado, lo que puede anticipar correcciones.
  - Indicadores como el "Investor Intelligence" y "Global Equity Flow" indican que el mercado está muy concentrado y tomando riesgos excesivos, señalando cautel